### 유투브 영상 요약하기

유투브에서 원하는 내용을 검색하고 텍스트로 가져와 언어 모델로 응용하는 방법을 다뤄보자.

In [1]:
!pip install youtube-search

  Using cached youtube_search-2.1.2-py3-none-any.whl.metadata (1.2 kB)
Using cached youtube_search-2.1.2-py3-none-any.whl (3.4 kB)


In [16]:
# 키워드 검색하기
from youtube_search import YoutubeSearch

videos = YoutubeSearch("What is A2A?", max_results=5).to_dict()
videos

[{'id': 'Sl9EZpE61xA',
  'thumbnails': ['https://i.ytimg.com/vi/Sl9EZpE61xA/hq720.jpg?sqp=-oaymwEjCOgCEMoBSFryq4qpAxUIARUAAAAAGAElAADIQj0AgKJDeAE=&rs=AOn4CLAI2p1dU9p9q97m8Tj2tyPewluofA',
   'https://i.ytimg.com/vi/Sl9EZpE61xA/hq720.jpg?sqp=-oaymwEXCNAFEJQDSFryq4qpAwkIARUAAIhCGAE=&rs=AOn4CLBpcJB5QG7qsRvGD7LAvzcWB_uUlg'],
  'title': 'What is A2A (Agent to Agent Protocol)? | A2A Explained',
  'long_desc': None,
  'channel': 'codebasics',
  'duration': '13:03',
  'views': '조회수 33,009회',
  'publish_time': '4개월 전',
  'url_suffix': '/watch?v=Sl9EZpE61xA&pp=ygUMV2hhdCBpcyBBMkE_'},
 {'id': 'WWHlehkRp3w',
  'thumbnails': ['https://i.ytimg.com/vi/WWHlehkRp3w/hq720.jpg?sqp=-oaymwEjCOgCEMoBSFryq4qpAxUIARUAAAAAGAElAADIQj0AgKJDeAE=&rs=AOn4CLDVSt8bALIgRKupS1h4d63Zr2dRxg',
   'https://i.ytimg.com/vi/WWHlehkRp3w/hq720.jpg?sqp=-oaymwEXCNAFEJQDSFryq4qpAwkIARUAAIhCGAE=&rs=AOn4CLDkdltB-XhkieSR0Oq1XmLoCRqrLA'],
  'title': "Google's A2A Protocol in 100 Seconds (AI Agents)",
  'long_desc': None,
  'channel': '

In [21]:
# 온전한 경로 만들기
video_url = 'https://youtube.com' + videos[3]['url_suffix']
video_url

'https://youtube.com/watch?v=GUozMSpnmcc&pp=ygUMV2hhdCBpcyBBMkE_'

#### YoutubeLoader 패키지로 유투브 자막 가져오기

In [4]:
!pip install youtube_transcript_api

  Using cached defusedxml-0.7.1-py2.py3-none-any.whl.metadata (32 kB)
Using cached defusedxml-0.7.1-py2.py3-none-any.whl (25 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [youtube_transcript_api]


In [32]:
from youtube_transcript_api import YouTubeTranscriptApi
video_id = "voaKr_JHvF4"
# ts = YouTubeTranscriptApi.list_transcripts(vid)
ytt_api = YouTubeTranscriptApi()
# ts = ytt_api.fetch(video_id, languages=['ko', 'en'])
# print(list(ts))

try:
    transcript_list = ytt_api.list(video_id)
    print(transcript_list)
    try:
        transcript = transcript_list.find_transcript(['ko']).fetch()
    except:
        transcript = transcript_list.find_transcript(['en']).fetch()
except Exception as e:
    print("자막을 가져올 수 없습니다:", e)
    transcript = []

# print(transcript)

# 2. transcript는 이런 리스트 형태
# [{'text': '안녕하세요', 'start': 0.0, 'duration': 4.2}, ...]
print(transcript[:2])

# 3. 텍스트 병합 (원하는 방식으로)
full_text = " ".join([t.text for t in transcript])

# 4. Document로 변환
from langchain.docstore.document import Document
doc = Document(
    page_content=full_text,
    metadata={
        "source": video_url,
        # "language": transcript.language if transcript else None,
        # "transcript": transcript  # 필요하면 원본 JSON도 저장
    }
)

print(doc)

For this video (voaKr_JHvF4) transcripts are available in the following languages:

(MANUALLY CREATED)
None

(GENERATED)
 - en ("English (auto-generated)")[TRANSLATABLE]

(TRANSLATION LANGUAGES)
 - ar ("Arabic")
 - zh-Hant ("Chinese (Traditional)")
 - nl ("Dutch")
 - fr ("French")
 - de ("German")
 - hi ("Hindi")
 - id ("Indonesian")
 - it ("Italian")
 - ja ("Japanese")
 - ko ("Korean")
 - pt ("Portuguese")
 - ru ("Russian")
 - es ("Spanish")
 - th ("Thai")
 - uk ("Ukrainian")
 - vi ("Vietnamese")
[FetchedTranscriptSnippet(text='The AI world is back with another', start=0.08, duration=3.599), FetchedTranscriptSnippet(text='three-letter acronym. Last time it was', start=1.68, duration=4.639)]
page_content='The AI world is back with another three-letter acronym. Last time it was MCP or the model context protocol. This time it's A2A from Google and it's the agent to agent protocol. Let's see what it is, how it works, and does it replace or complement MCP. Let's get right into [Music] it. 

In [34]:
# 길이가 1시간 이하인 영상만 고르기
print('총 영상 수:', len(videos))
# 영상 길이가 60분 이하인 영상만 남깁니다. 
videos = [v for v in videos if len(v['duration'].split(':')) < 3]
print('60분 이하 영상 수:', len(videos))

총 영상 수: 5
60분 이하 영상 수: 5


In [44]:
ytt_api = YouTubeTranscriptApi()
for v in videos:
    video_id = v['id']
    try:
        transcript_list = ytt_api.list(video_id)
        try:
            transcript = transcript_list.find_transcript(['ko']).fetch()
        except:
            transcript = transcript_list.find_transcript(['en']).fetch()
    except Exception as e:
        print("자막을 가져올 수 없습니다:", e)
        transcript = []

    # 3. 텍스트 병합 (원하는 방식으로)
    full_text = " ".join([t.text for t in transcript])

    # 4. Document로 변환
    from langchain.docstore.document import Document
    doc = Document(
        page_content=full_text,
        metadata={
            "source": video_url,
            # "language": transcript.language if transcript else None,
            # "transcript": transcript  # 필요하면 원본 JSON도 저장
        }
    )

    v['content'] = [doc]

videos

[{'id': 'Sl9EZpE61xA',
  'thumbnails': ['https://i.ytimg.com/vi/Sl9EZpE61xA/hq720.jpg?sqp=-oaymwEjCOgCEMoBSFryq4qpAxUIARUAAAAAGAElAADIQj0AgKJDeAE=&rs=AOn4CLAI2p1dU9p9q97m8Tj2tyPewluofA',
   'https://i.ytimg.com/vi/Sl9EZpE61xA/hq720.jpg?sqp=-oaymwEXCNAFEJQDSFryq4qpAwkIARUAAIhCGAE=&rs=AOn4CLBpcJB5QG7qsRvGD7LAvzcWB_uUlg'],
  'title': 'What is A2A (Agent to Agent Protocol)? | A2A Explained',
  'long_desc': None,
  'channel': 'codebasics',
  'duration': '13:03',
  'views': '조회수 33,009회',
  'publish_time': '4개월 전',
  'url_suffix': '/watch?v=Sl9EZpE61xA&pp=ygUMV2hhdCBpcyBBMkE_',
  'content': [Document(metadata={'source': 'https://youtube.com/watch?v=GUozMSpnmcc&pp=ygUMV2hhdCBpcyBBMkE_'}, page_content="Yesterday Google released agent to agent protocol and in this video I will provide a very intuitive explanation of what exactly it is. Obviously you need to know what are AI agents. If you don't I am linking couple of videos and here is a demo of Google agent space which uses A2A protocol. So le

#### 자막 내용 요약하기

In [35]:
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env")

True

In [36]:
# 언어 모델 설정
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")
model.invoke("안녕?")

AIMessage(content='안녕하세요! 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 10, 'total_tokens': 20, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-C4qcMj2eJSxeWWaH6nyNssZoAdLFP', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--fdaf9ddf-31c7-490d-9c18-cea2cdfb294c-0', usage_metadata={'input_tokens': 10, 'output_tokens': 10, 'total_tokens': 20, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [41]:
videos[0]['content']

Document(metadata={'source': 'https://youtube.com/watch?v=GUozMSpnmcc&pp=ygUMV2hhdCBpcyBBMkE_'}, page_content="Yesterday Google released agent to agent protocol and in this video I will provide a very intuitive explanation of what exactly it is. Obviously you need to know what are AI agents. If you don't I am linking couple of videos and here is a demo of Google agent space which uses A2A protocol. So let's say if you are an organization you are hiring a software engineer you can attach PDF file into this agent space and it will discover the agents which will help with the task. Okay so that is the first step in A2A protocol which is this particular client. Okay the the chatbot that you're seeing is able to discover other agents. So this chatbot itself is an agent but it is having this capability to discover other agents which can help it perform particular task. So it found this particular sourcing agent and then it is talking to that. Okay. So it is talking to that using a protocol. 

In [45]:
# 영상 하나만 요약하기
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

# 동영상 요약 프롬프트 작성
prompt = ChatPromptTemplate.from_messages(
    [("system", "다음 영상에 대한 요약을 한국어로 만들어줘 :\\n\\n{context}")]
)

chain = create_stuff_documents_chain(model, prompt)

result = chain.invoke({"context": videos[0]['content']})
result

'어제 Google이 에이전트 간 통신 프로토콜(A2A)을 출시했습니다. 이 비디오에서는 해당 프로토콜이 무엇인지에 대한 직관적인 설명을 제공합니다. AI 에이전트를 이해해야 하며, 이에 대한 소개 영상도 링크되어 있습니다.\n\nA2A 프로토콜은 한 에이전트가 다른 에이전트와 표준화된 방식으로 소통할 수 있게 해줍니다. 예를 들어, 조직에서 소프트웨어 엔지니어를 채용하는 경우, PDF 파일을 에이전트 공간에 첨부하면, 이 에이전트가 필요한 작업을 도와줄 다른 에이전트를 찾아냅니다. 이 챗봇은 다른 에이전트를 발견하고, 필요한 경우 명확한 질문을 하며 협력 과정을 거칩니다. \n\n이 프로토콜은 고객과 에이전트 간의 동적 상호작용을 지원하며, 여러 에이전트가 협력할 수 있도록 합니다. A2A 프로토콜은 AI 에이전트가 서로 통신하기 위한 표준 프로토콜로 정의되어 있습니다.\n\n이 비디오에서는 A2A와 모델 컨텍스트 프로토콜(MCP)의 차이를 설명하기 위해 자동차 수리사례를 예로 들고 있습니다. MCP는 LLM이 도구 및 자원과 통신하는 데 도움을 주는 프로토콜인 반면, A2A는 에이전트 간의 통신을 다룹니다.\n\nA2A 프로토콜의 필수 개념 중 하나는 에이전트 카드입니다. 고객이 적합한 수리사를 찾기 위해 에이전트 카드를 통해 필요한 정보를 검색하는 방식과 유사합니다. 다양한 에이전트를 발견하는 방법도 여러 가지가 있으며, 각각의 에이전트는 특정 형식의 에이전트 카드를 가지고 있습니다.\n\n이 프로토콜은 Google 및 여러 기업들에 의해 사용되고 있으며, 에이전트 간 상호 운용성을 보장하기 위해 표준화된 방식으로 에이전트를 소통하게 해줍니다. A2A 프로토콜은 HTTP 및 JSON RPC 등의 기존 프로토콜 위에 구축되어 있으며, 스트리밍 요구 사항 및 오류 처리도 지원합니다.\n\n영상은 A2A 프로토콜의 작동 방식을 설명하고, 코드 예제와 함께 에이전트의 기능을 보여줍니다. A2A의 출시는 인공지능 세계에 새로운 시대를 여는 중요한 발전으로 평가받고

In [46]:
# 모든 영상 요약하기
# 모든 비디오에 대해 요약을 생성
from tqdm import tqdm # tqdm은 진행 상황을 보여주는 라이브러리

for v in tqdm(videos):
    v['summary'] = chain.invoke({"context": v['content']})
   
videos

100%|██████████| 5/5 [00:49<00:00,  9.86s/it]


[{'id': 'Sl9EZpE61xA',
  'thumbnails': ['https://i.ytimg.com/vi/Sl9EZpE61xA/hq720.jpg?sqp=-oaymwEjCOgCEMoBSFryq4qpAxUIARUAAAAAGAElAADIQj0AgKJDeAE=&rs=AOn4CLAI2p1dU9p9q97m8Tj2tyPewluofA',
   'https://i.ytimg.com/vi/Sl9EZpE61xA/hq720.jpg?sqp=-oaymwEXCNAFEJQDSFryq4qpAwkIARUAAIhCGAE=&rs=AOn4CLBpcJB5QG7qsRvGD7LAvzcWB_uUlg'],
  'title': 'What is A2A (Agent to Agent Protocol)? | A2A Explained',
  'long_desc': None,
  'channel': 'codebasics',
  'duration': '13:03',
  'views': '조회수 33,009회',
  'publish_time': '4개월 전',
  'url_suffix': '/watch?v=Sl9EZpE61xA&pp=ygUMV2hhdCBpcyBBMkE_',
  'content': [Document(metadata={'source': 'https://youtube.com/watch?v=GUozMSpnmcc&pp=ygUMV2hhdCBpcyBBMkE_'}, page_content="Yesterday Google released agent to agent protocol and in this video I will provide a very intuitive explanation of what exactly it is. Obviously you need to know what are AI agents. If you don't I am linking couple of videos and here is a demo of Google agent space which uses A2A protocol. So le